In [ ]:
pip install torch_geometric

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch_geometric.data import HeteroData, DataLoader
from torch_geometric.nn import RGCNConv
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, to_hetero, GCNConv
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear
from torch_geometric.nn import global_mean_pool
from torch_geometric.loader import DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
nodes = pd.read_csv('test_nodes.csv')
edges = pd.read_csv('test_edges.csv')
playerFrameData = pd.read_csv('test_playerFrameData.csv')
playerEdges = pd.read_csv('test_playerEdges.csv')
graphData = pd.read_csv('test_graph_data.csv').T.rename(columns={0: 'roundNum', 1: 'floorSec', 2: 'team1AliveNum', 3: 'team2AliveNum', 4: 'CTwinsRound'})

In [ ]:
nodes['x'] = nodes['x'].astype('float32')
nodes['y'] = nodes['y'].astype('float32')
playerFrameData = playerFrameData.astype('float32')

In [ ]:
data = HeteroData()

# Create a sample heterogeneous graph with node, edge, and multiple graph-level features
data['player'].x = torch.tensor(playerFrameData.values)
data['map'].x = torch.tensor(nodes.values)

data['map', 'connected_to', 'map'].edge_index = torch.tensor(edges.T.values)
data['player', 'closest_to', 'map'].edge_index = torch.tensor(playerEdges.loc[ (playerEdges['roundNum'] == 1) & (playerEdges['floorSec'] == 0)][['playerId','closestId']].T.values)


# Define multiple graph-level features
data.y = {
    'roundNum': torch.tensor(graphData['roundNum'].iloc[0].astype('float32')),
    'floorSec': torch.tensor(graphData['floorSec'].iloc[0].astype('float32')),
    'team1AliveNum': torch.tensor(graphData['team1AliveNum'].iloc[0].astype('float32')),
    'team2AliveNum': torch.tensor(graphData['team2AliveNum'].iloc[0].astype('float32')),
    'CTwinsRound': torch.tensor(graphData['CTwinsRound'].iloc[0])
}

In [ ]:
data = data.to_homogeneous()
data

Data(
  y={
    roundNum=1.0,
    floorSec=0.0,
    team1AliveNum=5.0,
    team2AliveNum=5.0,
    CTwinsRound=0,
  },
  edge_index=[2, 214],
  x=[191, 44],
  node_type=[191],
  edge_type=[214]
)

In [ ]:
train_loader = DataLoader([data], batch_size=1, shuffle=True)

In [ ]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 1
DataBatch(
  y={
    roundNum=[1],
    floorSec=[1],
    team1AliveNum=[1],
    team2AliveNum=[1],
    CTwinsRound=[1],
  },
  edge_index=[2, 214],
  x=[191, 44],
  node_type=[191],
  edge_type=[214],
  batch=[191],
  ptr=[2]
)



In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(data.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 1)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(44, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(64, 1, bias=True)
)


In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y['CTwinsRound'])  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.


for epoch in range(1, 171):
    train()
    # train_acc = test(train_loader)
    # print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')

<IPython.core.display.Javascript object>